In [8]:
%run superuser-init.py --overwrite true

Posting group:  auai.org
Posting group:  auai.org/UAI
Posting group:  auai.org/UAI/2017
Posting group:  auai.org/UAI/2017/Program_Co-Chairs
Posting group:  auai.org/UAI/2017/Senior_Program_Committee
Posting group:  auai.org/UAI/2017/Senior_Program_Committee/invited
Posting group:  auai.org/UAI/2017/Senior_Program_Committee/declined
Posting group:  auai.org/UAI/2017/Senior_Program_Committee/emailed
Posting group:  auai.org/UAI/2017/Senior_Program_Committee/reminded
Posting group:  auai.org/UAI/2017/Program_Committee
Posting group:  auai.org/UAI/2017/Program_Committee/invited
Posting group:  auai.org/UAI/2017/Program_Committee/declined
Posting invitation: auai.org/UAI/2017/-/submission
Posting invitation: auai.org/UAI/2017/-/blind-submission
Posting invitation: auai.org/UAI/2017/-/spc_invitation
Posting invitation: auai.org/UAI/2017/-/spc_registration
Posting invitation: auai.org/UAI/2017/-/Paper/Metadata
Posting invitation: auai.org/UAI/2017/-/Reviewer/Metadata


## Set up instance

In [1]:
import openreview
import csv
import re
import requests
from collections import defaultdict

client = openreview.Client(username="OpenReview.net",password="OpenReview_beta",baseurl="http://localhost:3000")

def assign_reviewer(reviewer,paper_number,submissions):
    notes = [note for note in submissions if str(note.number)==str(paper_number)]
    
    valid_email = re.compile('^[^@\s,]+@[^@\s,]+\.[^@\s,]+$')
    valid_tilde = re.compile('~.+')
    reviewer_group = None
    if not notes:
        print "Paper number " + paper_number + " does not exist"
    elif not valid_email.match(reviewer) and not valid_tilde.match(reviewer):
        print "Program Committee Member \""+reviewer+"\" invalid. Please check for typos and whitespace."
    else:
        #need to incorporate conflicts. get them from public profile? confirm this with UAI documentation
        reviewer_group = get_reviewer_group(reviewer, paper_number, [])
    return reviewer_group

def get_reviewer_group(reviewer, paper_number, conflict_list):

    reviewers = client.get_group('auai.org/UAI/2017/Paper'+paper_number+'/Reviewers')
    nonreaders_reviewers = client.get_group('auai.org/UAI/2017/Paper'+paper_number+'/Reviewers/NonReaders')
    existing_reviewers = reviewers.members

    conference_reviewers = client.get_group(PC)
    if not (reviewer in conference_reviewers.members):
        client.add_members_to_group(conference_reviewers,reviewer)

    next_reviewer = get_next_reviewer_id(reviewer, paper_number)

    new_reviewer_id = 'auai.org/UAI/2017/Paper' + str(paper_number) + '/' + next_reviewer
    new_reviewer = create_reviewer_group(new_reviewer_id, reviewer, paper_number, conflict_list)

    client.add_members_to_group(reviewers,[reviewer])
    client.add_members_to_group(nonreaders_reviewers,[new_reviewer.id])
    
    return new_reviewer


def get_next_reviewer_id(reviewer, paper_number):

    headers = {
    'User-Agent': 'test-create-script',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + client.token
    }

    response = requests.get(client.baseurl + '/groups?id=auai.org/UAI/2017/Paper' + paper_number + '/AnonReviewer.*', headers=headers)
    reviewers = response.json()

    if reviewers:
        numbers = []
        for existing_reviewer in reviewers:

            if reviewer in existing_reviewer['members']:
                print "reviewer " + reviewer + " found in " + existing_reviewer['id']
                return None
            numbers.append(int(existing_reviewer['id'].split('AnonReviewer')[1]))

        return "AnonReviewer" + str(max(numbers) + 1)

    else:
        return "AnonReviewer1"




def create_reviewer_group(new_reviewer_id, reviewer, paper_number, conflict_list):
    new_reviewer = openreview.Group(
        new_reviewer_id,
        signatures=['auai.org/UAI/2017'],
        writers=['auai.org/UAI/2017'],
        members=[reviewer],
        readers=['auai.org/UAI/2017', COCHAIRS, SPC, PC],
        nonreaders=conflict_list,
        signatories=[new_reviewer_id])
    client.post_group(new_reviewer)
    return new_reviewer

### Generate dummy reviewers

In [ ]:
last = 'Reviewer'
password = '1234'

users = []
for first in ['Red', 'Orange', 'Yellow', 'Green', 'Blue', 'Indigo', 'Violet']:
    print first+" Reviewer"
    email = ('%sreviewer@gmail.com' % first.lower())
    users.append(client.create_dummy(email=email,first=first,last=last,password=password,secure_activation=False))

reviewers = [u['user']['profile']['id'] for u in users]

### Generate fake paper submissions

In [ ]:
#modify the submission invitation so that we can make easy test notes
uaisub = client.get_invitation('auai.org/UAI/2017/-/submission')
uaisub.reply['content']= {
    u'title': {
        u'description': u'Title of paper.',
        u'order': 1,
        u'required': True,
        u'value-regex': u'.{1,250}'
    }
}
client.post_invitation(uaisub)

#Create three sample notes
sample_papers = [None]*7
sample_papers[0] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content={'title':'Red Paper'},
                                readers=[u'auai.org/UAI/2017/Program_Co-Chairs'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )
sample_papers[1] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content={'title':'Orange Paper'},
                                readers=[u'auai.org/UAI/2017/Program_Co-Chairs'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )
sample_papers[2] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content={'title':'Yellow Paper'},
                                readers=[u'auai.org/UAI/2017/Program_Co-Chairs'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )
sample_papers[3] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content={'title':'Green Paper'},
                                readers=[u'auai.org/UAI/2017/Program_Co-Chairs'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )
sample_papers[4] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content={'title':'Blue Paper'},
                                readers=[u'auai.org/UAI/2017/Program_Co-Chairs'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )
sample_papers[5] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content={'title':'Indigo Paper'},
                                readers=[u'auai.org/UAI/2017/Program_Co-Chairs'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )
sample_papers[6] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content={'title':'Violet Paper'},
                                readers=[u'auai.org/UAI/2017/Program_Co-Chairs'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )

for note in sample_papers:
    print note.content['title']
    posted_note = client.post_note(note) 

In [ ]:
program_committee = client.get_group('auai.org/UAI/2017/Program_Committee')
program_committee.members = reviewers
client.post_group(program_committee)
print "added reviewers to Program Committee"

## Matching Demo

In [2]:
blind_submissions = client.get_notes(invitation='auai.org/UAI/2017/-/blind-submission')
titlesdict = {note.forum: note.content['title'] for note in blind_submissions}

In [ ]:
#Current schema
%run populate_papers_data.py --betas ../data/paper-betas.xml --reviewerscores ../data/paper-reviewer-scores.xml --paperscores ../data/paper-paper-scores.xml
%run populate_reviewer_data.py --alphas ../data/reviewer-alphas.xml --reviewerscores ../data/reviewer-reviewer-scores.xml --bidscores ../data/bids-by-reviewers.xml

In [3]:
#Proposed schema
%run populate_papers_data.py --betas ../data/paper-betas.xml --reviewerscores ../data/paper-reviewer-scores.xml --paperscores ../data/paper-paper-scores.xml --bidscores ../data/bids-by-papers.xml
%run populate_reviewer_data.py --alphas ../data/reviewer-alphas.xml --reviewerscores ../data/reviewer-reviewer-scores.xml --titles ../data/reviewer-titles.xml

creating paper metadata note
generating note S1lME6I8vg
generating note rygZ4T8IDl
generating note rklx4T8UDe
generating note Hye1E68IPl
generating note SkgCmaLUPl
generating note ryxpmaLIwe
generating note HygnmpLUvg
creating reviewer metadata note
generating note for ~Indigo_Reviewer1
generating note for ~Green_Reviewer1
generating note for ~Violet_Reviewer1
generating note for ~Yellow_Reviewer1
generating note for ~Blue_Reviewer1
generating note for ~Red_Reviewer1
generating note for ~Orange_Reviewer1


In [4]:
%run populate_paper_reviewer_assignment.py 

looping over paper_metadata_notes
Note:  <openreview.openreview.Note object at 0x107582d10>
Reviewer_info:  {u'source': u'TPMS', u'score': 1, u'reviewer': u'~Red_Reviewer1'}
Reviewer_info:  {u'source': u'TPMS', u'score': 0, u'reviewer': u'~Orange_Reviewer1'}
Reviewer_info:  {u'source': u'TPMS', u'score': 0, u'reviewer': u'~Yellow_Reviewer1'}
Reviewer_info:  {u'source': u'TPMS', u'score': 0, u'reviewer': u'~Green_Reviewer1'}
Reviewer_info:  {u'source': u'TPMS', u'score': 0, u'reviewer': u'~Blue_Reviewer1'}
Reviewer_info:  {u'source': u'TPMS', u'score': 0, u'reviewer': u'~Indigo_Reviewer1'}
Reviewer_info:  {u'source': u'TPMS', u'score': 0, u'reviewer': u'~Violet_Reviewer1'}
Reviewer_info:  {u'source': u'ReviewerBids', u'score': 1, u'reviewer': u'~Red_Reviewer1'}
Reviewer_info:  {u'source': u'ReviewerBids', u'score': 0.75, u'reviewer': u'~Orange_Reviewer1'}
Reviewer_info:  {u'source': u'ReviewerBids', u'score': 0, u'reviewer': u'~Yellow_Reviewer1'}
Reviewer_info:  {u'source': u'ReviewerBi

In [5]:
assignments = [(paper_forum_reviewers_dict[forum][0],paper_forum_number_dict[forum]) for forum in titlesdict.keys()]

In [6]:
assignments

[(u'~Yellow_Reviewer1', 3),
 (u'~Violet_Reviewer1', 7),
 (u'~Red_Reviewer1', 1),
 (u'~Green_Reviewer1', 4),
 (u'~Blue_Reviewer1', 5),
 (u'~Indigo_Reviewer1', 6),
 (u'~Orange_Reviewer1', 2)]

In [7]:
for reviewer, papernumber in assignments:
    new_reviewer = assign_reviewer(reviewer,str(papernumber),blind_submissions)
    print "Assigning %s to %s" % (reviewer,new_reviewer.id)


Assigning ~Yellow_Reviewer1 to auai.org/UAI/2017/Paper3/AnonReviewer1
Assigning ~Violet_Reviewer1 to auai.org/UAI/2017/Paper7/AnonReviewer1
Assigning ~Red_Reviewer1 to auai.org/UAI/2017/Paper1/AnonReviewer1
Assigning ~Green_Reviewer1 to auai.org/UAI/2017/Paper4/AnonReviewer1
Assigning ~Blue_Reviewer1 to auai.org/UAI/2017/Paper5/AnonReviewer1
Assigning ~Indigo_Reviewer1 to auai.org/UAI/2017/Paper6/AnonReviewer1
Assigning ~Orange_Reviewer1 to auai.org/UAI/2017/Paper2/AnonReviewer1


In [ ]:
paper_reviewer_score_dict